In [1]:
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-12.1.0"
os.environ["PATH"] = f"{os.environ['CUDA_HOME']}/bin:{os.environ['PATH']}"
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CUDA_HOME']}/lib64:{os.environ['LD_LIBRARY_PATH']}"
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CUDA_HOME']}/lib:{os.environ['LD_LIBRARY_PATH']}"
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CUDA_HOME']}/extras/CUPTI/lib64:{os.environ['LD_LIBRARY_PATH']}'"
os.environ["CUDAToolkit_ROOT_DIR"] = f"{os.environ['CUDA_HOME']}"
os.environ["CUDAToolkit_ROOT"] = f"{os.environ['CUDA_HOME']}"

os.environ["CUDA_TOOLKIT_ROOT_DIR"] = f"{os.environ['CUDA_HOME']}"
os.environ["CUDA_TOOLKIT_ROOT"] = f"{os.environ['CUDA_HOME']}"
os.environ["CUDA_BIN_PATH"] = f"{os.environ['CUDA_HOME']}"
os.environ["CUDA_PATH"] = f"{os.environ['CUDA_HOME']}"
os.environ["CUDA_INC_PATH"] = f"{os.environ['CUDA_HOME']}/targets/x86_64-linux"
os.environ["CFLAGS"] = f"-I{os.environ['CUDA_HOME']}/targets/x86_64-linux/include:{os.environ['CFLAGS']}"
os.environ["CUDAToolkit_TARGET_DIR"] = f"{os.environ['CUDA_HOME']}/targets/x86_64-linux"

In [1]:
import torch
import torchaudio
import torchaudio.functional as F
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

audio_path = Path("../downloads/wavs_16/PSA/PSA_019.wav")
json_path = Path("../data/openbible_swahili/PSA.json")
book, chapter = json_path.stem, audio_path.stem

In [2]:
import json
import re
import string
import unicodedata
from unidecode import unidecode
from num2words import num2words

def preprocess_verse(text: str) -> str:
    text = unidecode(text)
    text = unicodedata.normalize('NFKC', text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r"\d+", lambda x: num2words(int(x.group(0)), lang="sw"), text)
    text = re.sub("\s+", " ", text)
    return text

def load_transcripts(json_path, chapter):
    with open(json_path, "r") as f:
        data = json.load(f)
    
    # convert PSA 19:1 -> PSA_019
    get_chapter = lambda x: x.split()[0] + "_" + x.split(":")[0].split()[1].zfill(3)
    # filter by book and chapter
    transcripts = [d["verseText"] for d in data if get_chapter(d["verseNumber"]) == chapter]
    verse_ids = [d["verseNumber"] for d in data if get_chapter(d["verseNumber"]) == chapter]
    return verse_ids, transcripts

In [3]:
verse_ids, transcripts = load_transcripts(json_path, chapter)
verses = [preprocess_verse(v) for v in transcripts]
# insert "*" before every verse for chapter intro or verse number
augmented_verses = ["*"] * len(verses) * 2
augmented_verses[1::2] = verses

words = [verse.split() for verse in verses]
augmented_words = [word for verse in augmented_verses for word in verse.split()]

In [4]:
bundle = torchaudio.pipelines.MMS_FA
model = bundle.get_model(with_star=True).to(device)
LABELS = bundle.get_labels()
DICTIONARY = bundle.get_dict()

In [7]:
chunk_size_s = 15
waveform, sr = torchaudio.load(audio_path)
chunk_size_frames = chunk_size_s * sr
chunks = [waveform[:, i : i + chunk_size_frames] for i in range(0, waveform.shape[1], chunk_size_frames)]

In [14]:
emissions = []

with torch.inference_mode():
    for chunk in chunks:
        emission, _ = model(chunk.to(device))
        emissions.append(emission)

emission = torch.cat(emissions, dim=1)
assert len(DICTIONARY) == emission.shape[2]
num_frames = emission.size(1)

In [ ]:
# probs = torch.softmax(emission, dim=2)
# greedy_path = torch.argmax(probs, dim=-1).squeeze().cpu().numpy()
# predicted_tokens = [LABELS[i] == "*" for i in greedy_path]

In [17]:
def align(emission, tokens):
    targets = torch.tensor([tokens], dtype=torch.int32, device=device)
    alignments, scores = F.forced_align(emission, targets, blank=0)

    alignments, scores = alignments[0], scores[0]  # remove batch dimension for simplicity
    scores = scores.exp()  # convert back to probability
    return alignments, scores

def unflatten(list_, lengths):
    assert len(list_) == sum(lengths)
    i = 0
    ret = []
    for l in lengths:
        ret.append(list_[i : i + l])
        i += l
    return ret

def compute_alignments(emission, transcript, dictionary):
    tokens = [dictionary[char] for word in transcript for char in word]
    alignment, scores = align(emission, tokens)
    token_spans = F.merge_tokens(alignment, scores)
    word_spans = unflatten(token_spans, [len(word) for word in transcript])
    return word_spans

In [21]:
word_spans = compute_alignments(emission, augmented_words, DICTIONARY)

In [23]:
word_spans

[[TokenSpan(token=28, start=0, end=392, score=1.0)],
 [TokenSpan(token=10, start=392, end=393, score=0.7225428819656372),
  TokenSpan(token=17, start=395, end=396, score=0.9466070532798767),
  TokenSpan(token=2, start=399, end=400, score=0.9535767436027527),
  TokenSpan(token=4, start=404, end=405, score=0.9307804107666016),
  TokenSpan(token=14, start=407, end=408, score=0.9542909860610962),
  TokenSpan(token=6, start=409, end=410, score=0.9911168813705444)],
 [TokenSpan(token=23, start=418, end=419, score=0.9872608780860901),
  TokenSpan(token=2, start=420, end=421, score=0.983554482460022),
  TokenSpan(token=4, start=423, end=424, score=0.9824832677841187),
  TokenSpan(token=1, start=426, end=427, score=0.9853246808052063),
  TokenSpan(token=7, start=430, end=431, score=0.9898566007614136),
  TokenSpan(token=1, start=433, end=434, score=0.9946893453598022),
  TokenSpan(token=4, start=437, end=438, score=0.9732928276062012),
  TokenSpan(token=14, start=440, end=441, score=0.972795307

In [ ]:
from IPython.display import Audio

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)

def preview_word(waveform, spans, num_frames, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    segment = waveform[:, x0:x1]
    return Audio(segment.numpy(), rate=sample_rate)

In [ ]:
word_only_spans = [spans for spans, word in zip(word_spans, augmented_words) if word != "*"]
assert len(word_only_spans) == sum(len(word) for word in words)

In [ ]:
segments, labels = [], []
start = 0
for verse_words in words:
    end = start + len(verse_words)
    verse_spans = word_only_spans[start: end]
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * verse_spans[0][0].start)
    x1 = int(ratio * verse_spans[-1][-1].end)
    transcript = " ".join(verse_words)
    segment = waveform[:, x0: x1]
    start = end
    segments.append(segment)
    labels.append(transcript)

In [ ]:
assert len(segments) == len(verse_ids)

In [ ]:
from IPython.display import Audio

idx = 5
print(labels[idx])
Audio(segments[idx].numpy(), rate=bundle.sample_rate)

huchomoza upande mmoja wa mbingu na kufanya mzunguko wake hadi upande mwingine hakuna kilichojificha joto lake


In [ ]:
from pathlib import Path
from scipy.io.wavfile import write

output_dir = Path("../outputs/openbible_swahili/")
output_dir = output_dir / book / chapter
output_dir.mkdir(parents=True, exist_ok=True)

for verse_id, segment, label in zip(verse_ids, segments, labels):
    verse_number = verse_id.split(":")[-1].zfill(3)
    verse_file_name = chapter + "_" + verse_number
    audio_path = (output_dir / verse_file_name).with_suffix(".wav")
    transcript_path = (output_dir / verse_file_name).with_suffix(".txt")
    write(audio_path, bundle.sample_rate, segment.squeeze().numpy())
    with open(transcript_path, "w") as f:
        f.write(label)